## 簡単な例 (A simple example)



Scala kernelからのプログラム実行のため，まず以下の関数を定義する．



In [1]:
import scala.sys.process.Process
object bash {
  def !(cmd: String): Int = Process(Seq("bash", "-c", cmd)).!
  def !!(cmd: String): String = Process(Seq("bash", "-c", cmd)).!!
}
def out(fileName: String)(block: => Unit): Unit = {
  val o = new java.io.PrintStream(fileName)
  Console.withOut(o) { block }
  o.close
}
def neg(lit: String) = if (lit.startsWith("~")) lit.drop(1) else "~" + lit
def values(lits: Seq[String], model: String): Seq[Int] = {
  val m = model.split("\\s+").map(x => {
    if (x.startsWith("~")) x.drop(1) -> 0 else x -> 1
  }).toMap
  lits.map(lit => m.getOrElse(lit, m(neg(lit))))
}

$\textit{waerden}(3,3;9)$ が充足不能 (unsat)であることを，
入力ファイル [waerden-3-3-9.sat](../prog/waerden-3-3-9.sat) を用いて確かめる．



In [1]:
bash ! "sat13 < ~/prog/waerden-3-3-9.sat"

$\textit{waerden}(j,k;n)$ のCNF式を出力するプログラムは [sat-waerden](../knuth/pdf/sat-waerden.pdf) である．



In [1]:
bash ! "sat-waerden 3 3 8"

以下のようにすれば sat13 の入力として与えることができる．



In [1]:
bash ! "sat-waerden 3 3 8 | sat13"

同様のことを行うScalaの関数を定義する．



In [1]:
def waerden(j: Int, k: Int, n: Int): Unit = {
  for (d <- 1 to n; i <- 1 to n; if i+(j-1)*d <= n)
    println((i to i+(j-1)*d by d).mkString(" "))
  for (d <- 1 to n; i <- 1 to n; if i+(k-1)*d <= n)
    println((i to i+(k-1)*d by d).map("~" + _).mkString(" "))
}

以下のようにすれば sat13 の入力として与えることができる．



In [1]:
// waerden(3, 3, 8) の出力結果をファイル /tmp/a.sat に保存する
out("/tmp/a.sat") {
  waerden(3, 3, 8)
}
// /tmp/a.sat を入力として sat13 を実行する
bash ! "sat13 </tmp/a.sat"

結果を二進列に変換するには以下のようにする．



In [1]:
val (j,k,n) = (3,3,8)
val model = bash !! s"sat-waerden $j $k $n | sat13"
val xs = for (i <- 1 to n) yield i.toString
values(xs, model)

結果として `Vector(1, 0, 1, 0, 0, 1, 0, 1)` が得られ $(x_1x_2\cdots x_8)=(10100101)$ であることがわかる．

